In [1]:
import os
import socket
import subprocess
import pandas as pd
import io
from contextlib import redirect_stdout
import itertools
import warnings
from datetime import datetime


In [16]:
def islocal(): return socket.gethostname()=="rc-idsia-macBook.local"
def isserver(): return socket.gethostname()=="ec6120930b2d"
def strdate(): return datetime.today().strftime('%Y-%m-%d-%H-%M-%S')


In [3]:
if islocal():
    prj_path = "/Users/rcabanas/GoogleDrive/IDSIA/causality/dev/crema/CreMA"
    os.environ['JAVA_HOME'] = "/Library/Java/JavaVirtualMachines/openjdk-12.0.1.jdk/Contents/Home"
elif isserver():
    prj_path="/home/rcabanas/work_java_crema/crema/CreMA"
else:
    warnings.warn("Unknown server, set prj_path variable manually.")
os.chdir(prj_path)
!pwd

/Users/rcabanas/GoogleDrive/IDSIA/causality/dev/crema/CreMA


In [4]:
exp_folder = f"{prj_path}/papers/pgm20/experiments/"
log_folder = f"{exp_folder}/logs/"
res_folder = f"{exp_folder}/results/"
jarfile=f"{prj_path}/target/CreMA-0.1.1-SNAPSHOT-jar-with-dependencies.jar"

In [10]:
!mvn install:install-file -Dfile=./external/lpsolve55j.jar -DgroupId=net.sf.lpsolve -DartifactId=lp_solve -Dversion=5.5.2 -Dpackaging=jar

[INFO] Scanning for projects...
[WARNING] 
[WARNING] Some problems were encountered while building the effective model for ch.idsia:CreMA:jar:0.1.1-SNAPSHOT
[WARNING] 'dependencies.dependency.(groupId:artifactId:type:classifier)' must be unique: org.apache.commons:commons-math3:jar -> duplicate declaration of version 3.6.1 @ line 125, column 15
[WARNING] 
[WARNING] It is highly recommended to fix these problems because they threaten the stability of your build.
[WARNING] 
[WARNING] For this reason, future Maven versions might no longer support building such malformed projects.
[WARNING] 
[INFO] 
[INFO] ---------------------------< ch.idsia:CreMA >---------------------------
[INFO] Building Credal Model Algorithms 0.1.1-SNAPSHOT
[INFO] --------------------------------[ jar ]---------------------------------
[INFO] 
[INFO] --- maven-install-plugin:2.4:install-file (default-cli) @ CreMA ---
[INFO] Installing /home/rcabanas/work_java_crema/crema/CreMA/external/lpsolve55j.jar to /home/rcaba

In [11]:
!mvn install:install-file -Dfile=./external/polco.jar -DgroupId=ch.javasoft.polco -DartifactId=polco -Dversion=4.7.1 -Dpackaging=jar

[INFO] Scanning for projects...
[WARNING] 
[WARNING] Some problems were encountered while building the effective model for ch.idsia:CreMA:jar:0.1.1-SNAPSHOT
[WARNING] 'dependencies.dependency.(groupId:artifactId:type:classifier)' must be unique: org.apache.commons:commons-math3:jar -> duplicate declaration of version 3.6.1 @ line 125, column 15
[WARNING] 
[WARNING] It is highly recommended to fix these problems because they threaten the stability of your build.
[WARNING] 
[WARNING] For this reason, future Maven versions might no longer support building such malformed projects.
[WARNING] 
[INFO] 
[INFO] ---------------------------< ch.idsia:CreMA >---------------------------
[INFO] Building Credal Model Algorithms 0.1.1-SNAPSHOT
[INFO] --------------------------------[ jar ]---------------------------------
[INFO] 
[INFO] --- maven-install-plugin:2.4:install-file (default-cli) @ CreMA ---
[INFO] Installing /home/rcabanas/work_java_crema/crema/CreMA/external/polco.jar to /home/rcabanas/.

In [12]:
!mvn clean compile assembly:single

[INFO] Scanning for projects...
[WARNING] 
[WARNING] Some problems were encountered while building the effective model for ch.idsia:CreMA:jar:0.1.1-SNAPSHOT
[WARNING] 'dependencies.dependency.(groupId:artifactId:type:classifier)' must be unique: org.apache.commons:commons-math3:jar -> duplicate declaration of version 3.6.1 @ line 125, column 15
[WARNING] 
[WARNING] It is highly recommended to fix these problems because they threaten the stability of your build.
[WARNING] 
[WARNING] For this reason, future Maven versions might no longer support building such malformed projects.
[WARNING] 
[INFO] 
[INFO] ---------------------------< ch.idsia:CreMA >---------------------------
[INFO] Building Credal Model Algorithms 0.1.1-SNAPSHOT
[INFO] --------------------------------[ jar ]---------------------------------
[INFO] 
[INFO] --- maven-clean-plugin:2.5:clean (default-clean) @ CreMA ---
[INFO] Deleting /home/rcabanas/work_java_crema/crema/CreMA/target
[INFO] 
[INFO] --- maven-resources-plugi

In [19]:
def run_java(jarfile, javafile):
    cmd=f"java -cp {jarfile} {javafile}"
    result =subprocess.run(cmd, universal_newlines = True, shell=True,stdout = subprocess.PIPE)
    print(result.stdout)
    return result

def run_chain_markovian(N=4, exovarsize=6, method="CVE", seed=1234):
    javafile = f"{exp_folder}/ChainMarkovian.java {N} {exovarsize} {method} {seed}"
    #javafile = f"papers/pgm20/experiments/ChainMarkovian.java"

    result = run_java(jarfile, javafile)
    return [float(x) for x in result.stdout.splitlines()[-1].split(",")]

In [20]:
run_chain_markovian(N=3, exovarsize=6, method="CCALP")


ChainMarkovian: N=3 exovarsize=6 method=CCALP seed=1234
Running experiments...
Warm-up #0 in 1723.092 ms.
Warm-up #1 in 863.997 ms.
Measurement #0 in 767.744 ms. size=1.4766237805940996E-5
Measurement #1 in 685.946 ms. size=1.4766179653680211E-5
Measurement #2 in 574.513 ms. size=1.4766230691742877E-5
Measurement #3 in 543.24 ms. size=1.4879760955488486E-5
Measurement #4 in 538.339 ms. size=1.4855205414332495E-5
Measurement #5 in 539.827 ms. size=1.4766185015502309E-5
Measurement #6 in 535.734 ms. size=1.485521125244027E-5
Measurement #7 in 563.099 ms. size=1.498707381430986E-5
Measurement #8 in 508.843 ms. size=1.4871757153978038E-5
Measurement #9 in 519.392 ms. size=1.4766231677731945E-5
577.6677000000001,1.4828007343514749E-5



[577.6677000000001, 1.4828007343514749e-05]

In [21]:
def run_experiments(f, args, outkeys=("time", "size")):

    result = pd.DataFrame(columns=list(args.keys())+list(outkeys))
    log_file = f"{log_folder}{strdate()}_{f.__name__}.txt"

    for v in itertools.product(*list(args.values())):
        argsv = dict(zip(list(args.keys()), v))
        strio = io.StringIO()
        with open(log_file, 'a+') as logger:
            with redirect_stdout(strio):
                outvals = f(**argsv)
            logger.write(strio.getvalue())
        result = result.append({**argsv, **dict(zip(list(outkeys), outvals))}, ignore_index=True)
        print(dict(result.iloc[-1]))
    
    return result

In [22]:
args = {"N":[4,5], "exovarsize":[6], "seed":[1234], "method":["CVE", "CCVE","CCALP"]}
results = run_experiments(run_chain_markovian, args)

{'N': 4, 'exovarsize': 6, 'seed': 1234, 'method': 'CVE', 'time': 2.5734, 'size': 0.0}
{'N': 4, 'exovarsize': 6, 'seed': 1234, 'method': 'CCVE', 'time': 114.63489999999999, 'size': 0.0}
{'N': 4, 'exovarsize': 6, 'seed': 1234, 'method': 'CCALP', 'time': 1399.2522000000001, 'size': 3.234742328250828e-06}
{'N': 5, 'exovarsize': 6, 'seed': 1234, 'method': 'CVE', 'time': 4.3906, 'size': 0.0}
{'N': 5, 'exovarsize': 6, 'seed': 1234, 'method': 'CCVE', 'time': 150.0008, 'size': 0.0}
{'N': 5, 'exovarsize': 6, 'seed': 1234, 'method': 'CCALP', 'time': 2679.9761, 'size': 4.5914657665935275e-05}


In [23]:
results

,N,exovarsize,seed,method,time,size
0,4,6,1234,CVE,2.5734,0.000000
1,4,6,1234,CCVE,114.6349,0.000000
2,4,6,1234,CCALP,1399.2522,0.000003
3,5,6,1234,CVE,4.3906,0.000000
4,5,6,1234,CCVE,150.0008,0.000000
5,5,6,1234,CCALP,2679.9761,0.000046


In [24]:
res_file = f"{res_folder}{strdate()}_chain_markovian.csv"
results.to_csv(res_file)